<a href="https://colab.research.google.com/github/rnezil/NoHANDS/blob/main/NoHANDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NoHANDS: Non-Human/AI Neoliterature Detection System

Ryland Nezil, Uche Nwogbo, Alex Spacek

February 2024

In [120]:
# Install libraries
!pip install datasets

In [231]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.downloader as api
from datasets import load_dataset as ld
from datasets import Dataset as ds

# Download dataset
# id 0 to 1048107: human
# id 1048108 to 1418655: ai
train_dataset = ld('artem9k/ai-text-detection-pile', split='train[0:20000]+train[1050000:1070000]')
test_dataset = ld('artem9k/ai-text-detection-pile', split='train[20000:40000]+train[1070000:1090000]')
train_dataset = ds.from_dict({"data": train_dataset})
test_dataset = ds.from_dict({"data": test_dataset})

In [148]:
# Format data into list of lists of words
train, test, doc = [], [], []

train_size = 40000
test_size = 40000
doc_size = 40000

for i in range(train_size):
  train.append(list(train_dataset[i]['data']['text'].lower().split()))

for i in range(test_size):
  test.append(list(test_dataset[i]['data']['text'].lower().split()))

for i in range(doc_size):
  doc.append(list(train_dataset[i]['data']['text'].lower().split()))

In [149]:
# Function for creating tagged documents
def create_tagged_doc(paragraph):
  for i, sentence in enumerate(paragraph):
    yield gensim.models.doc2vec.TaggedDocument(sentence, [i])

In [150]:
# Document vectorization model
docvec_size = 200
trd = list(create_tagged_doc(doc))

model = gensim.models.doc2vec.Doc2Vec(vector_size=docvec_size, min_count=2, epochs=15)
model.build_vocab(trd)
model.train(trd, total_examples=model.corpus_count, epochs=model.epochs)

In [151]:
# Format training data
train_vec_array, test_vec_array = np.zeros(shape=(train_size,docvec_size)), np.zeros(shape=(test_size,docvec_size))
train_labels_array, test_labels_array = np.zeros(shape=(train_size,)), np.zeros(shape=(test_size,))
train_labels_array[-int(train_size/2):] = 1
test_labels_array[-int(test_size/2):] = 1

for i in range(train_size):
  train_vec_array[i,:] = model.infer_vector(train[i]).T

for i in range(test_size):
  test_vec_array[i,:] = model.infer_vector(test[i]).T

train_tensor = tf.convert_to_tensor(train_vec_array, dtype=tf.float32)
test_tensor = tf.convert_to_tensor(test_vec_array, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels_array, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels_array,dtype=tf.float32)

In [229]:
# AI generated text detection model
model2 = keras.Sequential([
    keras.layers.Input(shape=(docvec_size,)),
    keras.layers.Dense(134, activation='linear'),
    keras.layers.Dense(10, activation='sigmoid'),
    keras.layers.Dense(2),
])

optimizer = keras.optimizers.SGD(learning_rate=0.0005)
model2.compile(optimizer, loss='binary_crossentropy', metrics='binary_accuracy')

In [230]:
# Test the model
model2.fit(train_tensor, train_labels, epochs=7)
results = model2.evaluate(test_tensor, test_labels)

Epoch 1/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.5936 - binary_accuracy: 0.7033
Epoch 2/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.3331 - binary_accuracy: 0.9116
Epoch 3/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.2309 - binary_accuracy: 0.9466
Epoch 4/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.1785 - binary_accuracy: 0.9593
Epoch 5/7
1250/1250 [==============================] - 3s 3ms/step - loss: 0.1444 - binary_accuracy: 0.9670
Epoch 6/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.1234 - binary_accuracy: 0.9715
Epoch 7/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.1558 - binary_accuracy: 0.9640
